Dawit Reda
ID#10189420
Module 4 Web Scraping and APIs
MCIS 535 - Data Access and Transformation
Submitted to Dr. Daniel Wu

# Requirements

Write a Python program using mwviews API, BeautifulSoup, and other proper libraries to scrap web pages. The requirements are:
Scrap this web page to extract names of mathematicians listed on the web page.
Use the names extracted in step one and mwviews API to extract page hits from Wikimedia for each of the mathematicians.
Output the top 10 most popular mathematicians sorted by their Wikipedia page views count. You should also print out the number of mathematicians who has no entry in Wikipedia (as returned by mwviews API).
A sample output should look like this:

The Top 10 most popular mathematicians are:

Albert Einstein with 1089161 pageviews
Isaac Newton with 476031 pageviews
Aristotle with 343886 pageviews
Galileo Galilei with 330647 pageviews
Srinivasa Ramanujan with 305363 pageviews
René Descartes with 178436 pageviews
Archimedes with 146908 pageviews
Aryabhata with 139193 pageviews
John von Neumann with 134736 pageviews
Leonhard Euler with 101812 pageviews

But I cannot find results for 28 mathematicians on the lis

Here is the information how you can use mwviews API: https://github.com/mediawiki-utilities/python-mwviews.

In [2]:
#import all important libraries 
import pandas as pd
import requests
import urllib.request
import time
import csv
from requests import get
from contextlib import closing
from bs4 import BeautifulSoup
import itertools  
import numpy as np
from mwviews.api import PageviewsClient
import pageviewapi
import re
from tabulate import tabulate

In [3]:
#let's check the status of the get request
def get_Status_Code():
    url = 'http://www.fabpedigree.com/james/mathmen.htm'
    response = requests.get(url)
    #check the Status Codes
    if response:
        print('Success!')
    else:
        print('An error has occurred when the http request')

get_Status_Code()

Success!


In [4]:
#let's get the content of the web - using response.content or to convert the data into UTF-8 we can use .text as below
def get_requests(url):
    response = requests.get(url)
    return response.content

#let's pass the url of the Mathematicians to the function and get the content request 
url_Mathematicians = 'http://www.fabpedigree.com/james/mathmen.htm'
#get_requests(url_Mathematicians)

In [5]:
#let's pull the list of Mathematicians and put them in datafram
def list_of_Mathematicians():
    
    page = get_requests(url_Mathematicians)
    
    soup = BeautifulSoup(page, 'html.parser')

    list_Mathematicians=[]
    for link in soup.find_all('li'):
        name = link.find('a')
        list_Mathematicians.append(name.get_text('title'))
      
    return list_Mathematicians
    
 #let's get the list of  Mathematicians and length of the lists
print('Total number of Mathematicians from the page: ',len(list_of_Mathematicians()))
list_of_Mathematicians()[:10]
#list_of_Mathematicians().to_csv(r'Desktop\MathNames.csv')

Total number of Mathematicians from the page:  100


['Isaac Newton',
 'Archimedes',
 'Carl F. Gauss',
 'Leonhard Euler',
 'Bernhard Riemann',
 'Henri Poincaré',
 'Joseph-Louis Lagrange',
 'Euclid',
 'David Hilbert',
 'Gottfried W. Leibniz']

In [6]:
#get the full name of the Mathematicians with first name and last name concatinated with underscore
#Example:Isaac Newton will be Isaac_Newton 
def Mathematicians_full_name():
    Mathematicians = list_of_Mathematicians()
    first_and_last=[]
    list_of_math = [re.sub(r"[^a-zA-Z0-9]+", ' ', x) for x in Mathematicians]
    for i in range(len(Mathematicians)):
        Mathematicians_name =list_of_math[i]
        lists = Mathematicians_name.split()
        first_name =lists[0]
        last_name =lists[-1]
        if first_name==last_name:
            full_name=first_name
        else:
            full_name=first_name+'_'+last_name
        first_and_last.append(full_name)
    return first_and_last
 #let's check the ten Mathematicians full name
Mathematicians_full_name()[:10]

['Isaac_Newton',
 'Archimedes',
 'Carl_Gauss',
 'Leonhard_Euler',
 'Bernhard_Riemann',
 'Henri_Poincar',
 'Joseph_Lagrange',
 'Euclid',
 'David_Hilbert',
 'Gottfried_Leibniz']

Use the names extracted in step one and mwviews API to extract page hits from Wikimedia for each of the mathematicians.

In [7]:
def Top_10_most_popular_mathematicians():
    import operator
    Mathematicians = list_of_Mathematicians()
    
    p = PageviewsClient(user_agent='')
    #let's pull the last ten years page hit record for all Mathematicians listed above
    page_views = p.article_views("en.wikipedia", Mathematicians,  start="19990724", end="20190725")
    
    #create a dictionary to store the view hit of each Mathematicians
    dict_name_view = {mathematicians: 0 for mathematicians in Mathematicians }
    #loop over the page views result 
    for key in page_views.keys():
        for Mathematicians_key in page_views[key].keys():
            #concatinate first and last name of the mathematicians
            Mathematicians_Full_Name = Mathematicians_key.replace("_", " ")
            if page_views[key][Mathematicians_key]:
                dict_name_view[Mathematicians_Full_Name] += page_views[key][Mathematicians_key]
                

    #let's sort the result with the value of the dict
    sorted_Final_result = [(k, dict_name_view[k]) for k in sorted(dict_name_view, key=dict_name_view.get, reverse=True)]
    
    #convert the result into a dataframe
    df = pd.DataFrame.from_dict(sorted_Final_result)
    df.rename(columns={0:'Mathematicians Name',1:'Number of View'}, inplace=True)
    
    return df
    
Top_10_most_popular_mathematicians().head(10)    

,Mathematicians Name,Number of View
0,Albert Einstein,30924167
1,Isaac Newton,15123943
2,Srinivasa Ramanujan,12599757
3,Aristotle,10130664
4,Galileo Galilei,9702939
5,René Descartes,5244102
6,Pythagoras,4950907
7,Aryabhata,4867755
8,Archimedes,4560860
9,John von Neumann,3798992


In [8]:
#print out The Top 10 most popular mathematicians for the last ten years are:
def Final_output():
    df = Top_10_most_popular_mathematicians()
    print('\nThe Top 10 most popular mathematicians are:\n')
    for a in df[:10].index:
        print(df['Mathematicians Name'][a],' with ', df['Number of View'][a],' pageviews') 
    #let's add for the mathematicians with zero views by iterating in each rows of the mathematicians
    zero_views_result= df[df['Number of View'] == 0]
    print('\nBut I cannot find results for' , len(zero_views_result), 'mathematicians on the list')    

Final_output()        


The Top 10 most popular mathematicians are:

Albert Einstein  with  30924167  pageviews
Isaac Newton  with  15123943  pageviews
Srinivasa Ramanujan  with  12599757  pageviews
Aristotle  with  10130664  pageviews
Galileo Galilei  with  9702939  pageviews
René Descartes  with  5244102  pageviews
Pythagoras  with  4950907  pageviews
Aryabhata  with  4867755  pageviews
Archimedes  with  4560860  pageviews
John von Neumann  with  3798992  pageviews

But I cannot find results for 18 mathematicians on the list


### Reference

1. https://wikitech.wikimedia.org/wiki/Analytics/AQS/Pageviews
2. https://github.com/mediawiki-utilities/python-mwviews
3. http://www.fabpedigree.com/james/mathmen.htm